In [2]:
import numpy as np
import qutip as qt
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets
import random
%matplotlib inline

# Gaussian Noise simulation

Define a hamiltonian as 
$$
    H(t) = - \frac{1}{2} \omega \sigma_z - \frac{1}{2} Z_t \sigma_z, 
$$
where $Z_t$ is a Gaussian random variable $Z_t \sim \mathcal{N}(\mu, s^2)$

We solve the von-nueman equation
$$
    \dot{\rho} = -i [H(t), \rho],
$$

where $\rho$ is an input pure state and the frequency $f = \omega/2\pi$ and the mean $\mu$ and standard deviation $s$ of $Z_t$ are degrees of freedom. 

In [22]:
def H(t, freq, samples):
    """
    return H given a time, frequency, and a list of samples of Z_t
    """
    return (-np.pi*freq*qt.sigmaz() - 0.5*samples[int(t)]*qt.sigmaz())

def CBz(t, ain):
    """
    callback functino for qt.mesolve 
    """
    return H(t, ain[0], ain[1])


def SolveAndPlot(T, alpha, phi, freq, mu, sd):
    """
    Solve the von Neuman equation for gaussian phase noise with provided parameters 
    """
    # time vector
    tvec = np.arange(0,T)
    
    # sample Gaussian distribution
    samples = []
    for _ in tvec:
        samples.append(random.gauss(mu, sd))
    
    # calculate \psi_0
    beta = np.sqrt(1-alpha**2)
    psi0 = alpha*qt.basis(2,0) + beta*np.exp(-1j*2*np.pi*phi/360)*qt.basis(2,1)

    # solve
    ExpVec = qt.mesolve(CBz, psi0, tvec, [], [qt.sigmax(), qt.sigmay(), qt.sigmaz()], args=(freq, samples))
    PsiVec = qt.mesolve(CBz, psi0, tvec, [], [], args=(freq, samples))
    
    # plot
    fig, ax = plt.subplots()
    ax.plot(ExpVec.expect[0]);
    ax.plot(ExpVec.expect[1]);
    ax.plot(ExpVec.expect[2]);
    ax.set_xlabel('Time');
    ax.set_ylabel('Expectation values');
    ax.legend(("Sigma-X", "Sigma-Y", "Sigma-Z"));
    # ax.set_box_aspect((1, 1, 1)) 
    # ax = a3.Axes3D(plt.figure(figsize=(15,15))) 
    plt.show(fig)

    # Bloch plots 
    b = qt.Bloch()
    nrm = mpl.colors.Normalize(0,T)
    colors = cm.cool(nrm(tvec))
    b.vector_color = list(colors)
    for s in PsiVec.states:
        b.add_states(s)

    b.show()

In [23]:
_T = FloatSlider(min=1, max=100, step=1)
_DeltaT = FloatSlider(min=1, max=400, step=1)

_alpha = FloatSlider(min=0, max=1, step=0.05)
_phi = FloatSlider(min=0, max=360, step=1)
_freq = FloatSlider(min=0, max=300, step=0.5)
_mu = FloatSlider(min=-20, max=20, step=1)
_sd = FloatSlider(min=0, max=10, step=0.5)


interact_manual(SolveAndPlot, T=_T, DeltaT=_DeltaT, alpha=_alpha, phi=_phi, freq=_freq, mu=_mu, sd=_sd)

interactive(children=(FloatSlider(value=1.0, description='T', min=1.0, step=1.0), FloatSlider(value=0.0, descr…

<function __main__.SolveAndPlot(T, alpha, phi, freq, mu, sd)>